# Graph Representation with Lab Data only in Node Features: Directed Mixed Edge Definition
Implements & generates a patient graph representation using Demographics and Lab Variables. Lab Variables both influence patient similarity using DTW on each variable and are normalized/concatenated to node features (after cosine similarity for nodes is calculated). Uses tenfold Cross Validation to generate ten separate Data Objects, one for each iteration.

'Mixed' combines both knn and edge threshold by only adding the top k nearest neighbors if their similarity is above a flat threshold. 


This notebook generates the data objects that represent the patient graph network. After running this file, run the notebooks found in 'src/Model Train and Eval/' on the specified root folder defined by 'root_path' that includes your models in the 'processed' subfolder.

In [1]:
# Sets seed for Pytorch Lightning, sklearn, numpy, etc.
SEED = 1

# defines number of neighbors for mixed (n_neighbors with flat threshold)
n_neighbors = 5
min_threshold = 0.65

# defines whether to use shortened lab time series data (6 values per patient) or regular (12 per patient)
shortened = True

# root path to process raw dataset
root_path = '../../data/lab-oversampled-mixed5,0.65-noDTW'

In [2]:
## Standard libraries
import os
import os.path as osp
from os.path import exists
import numpy as np 
import pandas as pd
from dtw import *

## Imports for plotting
import matplotlib.pyplot as plt

## PyTorch
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

# Torchvision
import torchvision
from torchvision import transforms

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

from torch_geometric.data import Dataset, download_url, Data
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE

# torch geometric
try: 
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version. 
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details 
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric 
    import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

pl.seed_everything(SEED)
np.random.seed(SEED)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



Global seed set to 1


cuda:0


In [3]:
# removes all STUDY_ID indices that are not shared by BOTH df1
# and df2. function assumes STUDY_ID is the index value in both
# DataFrames.
# Returns: modified df1, modified df2. 
def removeNonIntersections(df1, df2):
    df1_index = df1.index.unique()
    # modify df2 based on df1's index
    df2 = df2[df2.index.isin(df1_index)]
    # modify df1 based on df2's new index
    df2_index = df2.index.unique()
    df1 = df1[df1.index.isin(df2_index)]
    return df1, df2


# takes in variables DataFrame & separates it into the 3 separate 
# DataFrames for each variable, also resetting the index value.
# Returns: separate (num_patient x 12) dataframes for gluc, ldl, 
# & hemo, respectively.
def separateVars(vars_df):
    gluc_df, ldl_df, hemo_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    gluc_row, ldl_row, hemo_row = [], [], [] # <- variables for storing the time series for a given patient
    prev_index = vars_df.index[0]
    for index, row in vars_df.iterrows():
        if index == prev_index:
            # add new points to each row
            gluc_row += [row['glucose_lab']]
            ldl_row += [row['ldl_lab']]
            hemo_row += [row['hemo_lab']]
        else: 
            # reset for new STUDY_ID, add each row to each respective DataFrame
            prev_index = index
            gluc_df = pd.concat([gluc_df, pd.DataFrame([gluc_row])], axis=0)
            ldl_df = pd.concat([ldl_df, pd.DataFrame([ldl_row])], axis=0)
            hemo_df = pd.concat([hemo_df, pd.DataFrame([hemo_row])], axis=0)
            
            gluc_row = [row['glucose_lab']]
            ldl_row = [row['ldl_lab']]
            hemo_row = [row['hemo_lab']]
    # add last row to df
    gluc_df = pd.concat([gluc_df, pd.DataFrame([gluc_row])], axis=0)
    ldl_df = pd.concat([ldl_df, pd.DataFrame([ldl_row])], axis=0)
    hemo_df = pd.concat([hemo_df, pd.DataFrame([hemo_row])], axis=0)
    # reset indices for each df
    gluc_df = gluc_df.reset_index(drop=True)
    ldl_df = ldl_df.reset_index(drop=True)
    hemo_df = hemo_df.reset_index(drop=True)
    
    return gluc_df, ldl_df, hemo_df

In [4]:
# separates DataFrame into train, test, and validate dfs
# based on SEED & user-defined proportion of train, test split.
# Returns: train_df, test_df, val_df
def train_test_val(df, train_size=0.7, test_size=0.2, SEED=0):
    train_df, test_df = train_test_split(df, train_size=train_size, random_state=SEED)
    test_df, val_df = train_test_split(test_df, train_size=(test_size / (1 - train_size)), random_state=SEED)
    return train_df, test_df, val_df


# runs train_test_val on a list of dfs, adding them to the output dictionary based 
# on their respective index's name (as a str) in name_list.
# Returns: train/test/val dictionaries containing each df's result; specify the string 
# name for the df to get the respective df mask.
def run_train_test_val(df_list, name_list, train_size=0.7, test_size=0.2, SEED=0):
    train_splits = {}
    test_splits = {}
    val_splits = {}
    for i in range(len(df_list)):
        train_df, test_df, val_df = train_test_val(df_list[i], train_size=train_size, test_size=test_size, SEED=SEED)

        train_splits[name_list[i]] = train_df
        test_splits[name_list[i]] = test_df
        val_splits[name_list[i]] = val_df
    return train_splits, test_splits, val_splits

In [5]:
# uses SMOTE to oversample a given X_df and y_df based on SEED.
# Returns: oversampled X_df, y_df.
def oversampleSMOTE(X_df, y_df, SEED=0):
    smote = SMOTE(random_state=SEED)
    return smote.fit_resample(X_df, y_df)


# uses SMOTE on a dictionary of X_dfs and one y_df.
# Returns: dictionary of oversampled X_dfs, oversampled y_df
def oversampleSMOTE_dict(dict_X_df, y_df, SEED=0):
    oversamp_dict = {}
    smote = SMOTE(random_state=SEED)
    for key in dict_X_df:
        temp_y_df = y_df.copy()
        oversamp_dict[key], temp_y_df = smote.fit_resample(dict_X_df[key], temp_y_df)
    
    return oversamp_dict, temp_y_df

In [6]:
# Takes in the train, test, & validate dicts & concatenates
# them into a new dictionary featuring all nodes.
# Returns: dictionary containing ALL x nodes, ALL y nodes, etc.
def concatSplits(train_splits, test_splits, val_splits):
    splits = {}
    # note: function assumes that all dicts have the same keys
    for key in train_splits:
        splits[key] = pd.concat((train_splits[key], test_splits[key], val_splits[key]), ignore_index=True)
    return splits

In [7]:
# takes in a df with time series data and returns a np 2d 
# array of the dtw distances, displaying a loading value
# with label load_str. 
# Returns: (len(df) x len(df)) np array with dtw distances.
def dtwDistance(df, load_str):
    dtw_distance = np.zeros([len(df), len(df)])
    x = 0
    for i1, row1 in df.iterrows():
        print(load_str + ':  ' + str(x) + ' / ' + str(len(df)), end='\r')
        for i2, row2 in df.iterrows():
            # get all 3 variables & caclulate each dtw

            if (i1 == i2):
                break

            val = dtw(row1.tolist(), row2.tolist(), keep_internals=True).distance
            dtw_distance[i1][i2] = val
            dtw_distance[i2][i1] = val
        x += 1
    print(load_str + ':  done!              ')
    return dtw_distance
    
    
# uses dtwDistance to calculate the distance similarity matrix
# for a given df, displaying a loading value with label
# load_str.
# Returns: (len(df) x len(df)) np array with dtw similarities
def dtwSimilarity(df, load_str):
    dtw_distance = dtwDistance(df, load_str)
    
    # get max and min value from dtw_distance to perform min-max normalization
    # Note: dtw_similarity is defined by 1 - (min-max normalization of dtw_distance)
    min_arr = np.zeros(len(dtw_distance))
    max_arr = np.zeros(len(dtw_distance))
    for i in range(len(dtw_distance)):
        row = np.delete(np.copy(dtw_distance[i]), i) # only considers non-major diagonal entries
        min_arr[i] = np.amin(row)
        max_arr[i] = np.amax(row)
    max_value = np.amax(max_arr)
    min_value = np.amin(min_arr)
    
    # set major diagonal to 0
    dtw_similarity = 1 - ((dtw_distance - min_value) / (max_value - min_value))
    for i in range(len(dtw_similarity)):
        dtw_similarity[i][i] = 0
    return dtw_similarity

In [8]:
# takes in df for node features and computes cosine
# similarity matrix, showing a loading bar with label
# load_str.
# Returns: (len(df) x len(df)) cosine similarity np array
def cosSimilarity(df, load_str):
    cos_similarity = np.zeros([len(df), len(df)])
    x = 0
    for index, row1 in df.iterrows():
        print(load_str + ':  ' + str(x) + ' / ' + str(len(df)), end='\r')
        row_similarities = np.zeros([1, len(df)])
        row_s_index = 0
        for i, row2 in df.iterrows():
            if (index == i):
                break
            # convert row and r into np arrays
            row1_arr = row1.to_numpy()
            row2_arr = row2.to_numpy()
            # calculate row similarity
            dot_product = np.dot(row1_arr, row2_arr) / (np.linalg.norm(row1_arr) * np.linalg.norm(row2_arr))
            cos_similarity[x][row_s_index] = dot_product
            cos_similarity[row_s_index][x] = dot_product
            row_s_index += 1
        x += 1
    print(load_str + ':  done!               ', end='\r')
    return cos_similarity


# takes in similarity matrix and converts it to an edge
# index based on a flat threshold.
# Returns: 2 x (# of edges) edge index np array
def toEdgeIndex_threshold(similarity, threshold=0.5):
    edge_start = []
    edge_end = []
    for row in range(len(similarity)):
        for col in range(len(similarity[row])):
            if similarity[row][col] >= threshold and row != col:
                # create 2 by 1 np array and concatenate to main one
                edge_start += [row]
                edge_end += [col]
    return [edge_start, edge_end]


# uses k-Nearest Neighbors with a minimum threshold to 
# define the edge index based on a set # of neighbors 
# per node and defined min_threshold.
# Returns: 2 x (2 * num_neighbors * # of nodes) edge index np array
def toEdgeIndex_mixed(similarity, num_neighbors=5, min_threshold=0.75):
    edge_start, edge_end = [], []
    for row in range(len(similarity)):    # num_neighbors, so a lot of nodes will have > num_neighbors neighbors)
        # find top N nodes to form edges with
        top_n_neighbors = np.argpartition(similarity[row], -num_neighbors)[-num_neighbors:]
        for end in top_n_neighbors:
            if similarity[row][end] >= min_threshold:
                edge_start += [row]
                edge_end   += [end]
    return [edge_start, edge_end]

In [9]:
# performs min-max normalization on time series df.
# Returns: normalized time series df.
def minMaxNormalize(df):
    min_arr = np.zeros(len(df.columns))
    max_arr = np.zeros(len(df.columns))
    for col in df:
        min_arr[col] = df.loc[df[col].idxmin()][col]
        max_arr[col] = df.loc[df[col].idxmax()][col]
    min_value = np.amin(min_arr)
    max_value = np.amax(max_arr)
    return (df.select_dtypes(include=['float64']) - min_value) / (max_value - min_value)

In [10]:
# takes in a list of dfs, a list of names for the dfs, and 
# the train/test/val index arrays and creates dictionaries
# for train/test/val.
# Returns: train df dict, test df dict, val df dict
def train_test_val_index(df_list, name_list, train_indices, test_indices, val_indices):
    train_splits = {}
    test_splits = {}
    val_splits = {}
    for i in range(len(df_list)):
        train_df, test_df, val_df = df_list[i].iloc[train_indices], df_list[i].iloc[test_indices], df_list[i].iloc[val_indices]
        train_splits[name_list[i]] = train_df
        test_splits[name_list[i]] = test_df
        val_splits[name_list[i]] = val_df
    return train_splits, test_splits, val_splits

In [11]:
# generates train/test/val index arrays for tenfold
# cross validation. 
# Returns: (10 x (train block size * size of block)) train index array, 
#          (10 x (test block size * size of block)) test index array, 
#          (10 x (val block size * size of block)) val index array, 
def generate_split_indices(node_len, k_fold=10, train_size=7, test_size=2, val_size=1, SEED=0):
    np.random.seed(SEED)
    indices = np.array(range(node_len))
    # shuffle indices, pop extra indices into seperate array
    np.random.shuffle(indices)
    indices_remain = indices[-(node_len % k_fold):]
    indices = indices[:-(node_len % k_fold)]
    # double the length to simplify looping of block categories & split it into 10 (* 2) buckets
    indices_double = np.concatenate((indices, indices))
    ind_double_list = np.split(indices_double, k_fold * 2)
    # concatenate remaining values onto first block
    ind_double_list[0] = np.concatenate((ind_double_list[0], indices_remain))
    
    train_splits, test_splits, val_splits = [], [], []
    for train_block_start in range(k_fold):
        # generate the index arrays for the ith start
        train_split = np.array(ind_double_list[train_block_start])
        test_split = np.array(ind_double_list[train_block_start + train_size])
        val_split = np.array(ind_double_list[train_block_start + train_size + test_size])
        for i in range(1, train_size):
            train_split = np.concatenate((train_split, ind_double_list[train_block_start + i]))
        for i in range(1, test_size):
            test_split = np.concatenate((test_split, ind_double_list[train_block_start + train_size + i]))
        for i in range(1, val_size):
            val_split = np.concatenate((val_split, ind_double_list[train_block_start + train_size + test_size + i]))
        train_splits += [train_split]
        test_splits += [test_split]
        val_splits += [val_split]
    return train_splits, test_splits, val_splits


In [12]:
# generates ten data objects for the TenfoldDataset that will
# later be initialized with the specified root. Each data object
# saved into the specified root directory will later be trained /
# tested.
# Returns: nothing but saves the 10 crossVal data objects in the
# processed data folder in the root database. 
def tenfold_cross_val(root=None, train_num=7, test_num=2, val_num=1, SEED=0):
    np.random.seed(SEED)
    # type check: make sure train/test/block sizes are positive and add to 10, root is specified
    if train_num <= 0 or test_num <= 0 or val_num <= 0 or train_num + test_num + val_num != 10:
        raise Exception("Block sizes must all be positive and add to 10.")
    if root == None:
        raise Exception("Please specify a root directory.")

    print('importing x, y, and lab variables, removing non-intersecting patients...')
    x_df = pd.read_csv(root + '\\raw\\x_pd.csv', index_col=0)
    y_df = pd.read_csv(root + '\\raw\\y_pd.csv', index_col=0)
    
    vars_path = root + '\\raw\\imputed_lab' + '_shortened.csv' if shortened else '.csv'
    
    vars_df = pd.read_csv(vars_path, index_col=1)
    vars_df = vars_df.drop(columns=['Unnamed: 0'])
    x_df, vars_df = removeNonIntersections(x_df, vars_df)
    y_df, vars_df = removeNonIntersections(y_df, vars_df)

    print('splitting variables into gluc, ldl, & hemo; resetting index and randomizing buckets...')
    # separate variables into separate gluc, ldl, and hemo dfs; reset index values:
    gluc_df, ldl_df, hemo_df = separateVars(vars_df)
    x_df = x_df.reset_index(drop=True)
    y_df = y_df.reset_index(drop=True)

    # calculate train/test/val indices for tenfold CV
    train_indices, test_indices, val_indices = generate_split_indices(len(x_df), SEED=SEED)
    
    
    # specify block indices for cross validation:
    indices = np.array(range(len(x_df)))
    np.random.shuffle(indices)
    
    # calculate the data object for each train-test-val split in the tenfold CV:
    for i in range(10):
        print('\n\nstarting data calculation ' + str(i) + ':\n----------')
        
        # check that file has not yet been created
        if exists(root + '\\processed\\data_' + str(i) + '.pt'):
            print('file already exists. Advancing to next block data generation...')
            continue

        # create separate dfs for train/test/val:
        df_list = [x_df, y_df, gluc_df, ldl_df, hemo_df]
        name_list = ['x', 'y', 'gluc', 'ldl', 'hemo']
        train_splits, test_splits, val_splits = train_test_val_index(df_list, name_list, train_indices[i], 
                                                                     test_indices[i], val_indices[i])
        # oversample training data
        y_train = train_splits.pop('y')
        train_splits, y_train = oversampleSMOTE_dict(train_splits, y_train, SEED=SEED)
        train_splits['y'] = y_train

        # create train/test/val masks, create overall dictionary of all nodes/features TODO: decompose this!
        train_mask = [1] * len(train_splits['x']) + [0] * (len(test_splits['x']) + len(val_splits['x']))
        test_mask = [0] * len(train_splits['x']) + [1] * len(test_splits['x']) + [0] * len(val_splits['x'])
        val_mask = [0] * (len(train_splits['x']) + len(test_splits['x'])) + [1] * len(val_splits['x'])
        splits = concatSplits(train_splits, test_splits, val_splits)
        print('Number of nodes after oversampling:', len(splits['x']))

#         # calculate DTW distance
#         dtw_similarity_gluc = dtwSimilarity(splits['gluc'], 'Calculating DTW Similarity matrix for Glucose')
#         dtw_similarity_ldl = dtwSimilarity(splits['ldl'], 'Calculating DTW Similarity matrix for LDL')
#         dtw_similarity_hemo = dtwSimilarity(splits['hemo'], 'Calculating DTW Similarity matrix for Hemoglobin')
        
        
        # min-max normalize lab data
        splits['gluc'] = minMaxNormalize(splits['gluc'])
        splits['ldl'] = minMaxNormalize(splits['ldl'])
        splits['hemo'] = minMaxNormalize(splits['hemo'])

        # append lab data to x (before calculating cosine similarity)
        splits['x'] = pd.concat((splits['x'], splits['gluc'], splits['ldl'], splits['hemo']), axis=1, ignore_index=True)
        cos_similarity = cosSimilarity(splits['x'], 'Calculting Cosine Similarity for non-lab metrics')

        # calculate cosine similarity, general similarity, & generate edge index
        similarity = cos_similarity # (1 - dtw_weight) * cos_similarity + dtw_weight * (dtw_similarity_gluc + dtw_similarity_ldl + dtw_similarity_hemo) / 3
        edge_index = toEdgeIndex_mixed(similarity, num_neighbors=n_neighbors, min_threshold=min_threshold)

        # Create Data object
        data = Data(x=torch.Tensor(splits['x'].to_numpy()), edge_index = torch.LongTensor(edge_index))
        data.y = torch.tensor(splits['y']['SARCOPENIA'].to_numpy(), dtype=torch.long)
        data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
        data.test_mask = torch.tensor(test_mask, dtype=torch.bool)
        data.val_mask = torch.tensor(val_mask, dtype=torch.bool)
        data.num_classes = 2

        torch.save(data, root + '\\processed\\' + f'data_{i}.pt')
    print('\n------------------------\ndone!')

In [13]:
tenfold_cross_val(root=root_path, SEED=SEED)

importing x, y, and lab variables, removing non-intersecting patients...
splitting variables into gluc, ldl, & hemo; resetting index and randomizing buckets...


starting data calculation 0:
----------
Number of nodes after oversampling: 1260
Calculting Cosine Similarity for non-lab metrics:  done!               

starting data calculation 1:
----------
Number of nodes after oversampling: 1252
Calculting Cosine Similarity for non-lab metrics:  done!               

starting data calculation 2:
----------
Number of nodes after oversampling: 1244
Calculting Cosine Similarity for non-lab metrics:  done!               

starting data calculation 3:
----------
Number of nodes after oversampling: 1258
Calculting Cosine Similarity for non-lab metrics:  done!               

starting data calculation 4:
----------
Number of nodes after oversampling: 1234
Calculting Cosine Similarity for non-lab metrics:  done!               

starting data calculation 5:
----------
Number of nodes after oversa